In [130]:
import os
import shutil
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
from sklearn.model_selection import train_test_split
import re
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score

tf.get_logger().setLevel('ERROR')


In [2]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42



In [3]:
text=pd.read_csv(r'C:\Users\Dell\Downloads\hate-speech-and-offensive-language-master\hate-speech-and-offensive-language-master\data\labeled_data.csv')

### Load the dataset

In [4]:
text=text.drop(columns=['count','offensive_language','neither'])

In [5]:
text

,Unnamed: 0,hate_speech,class,tweet
0,0,0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...
24778,25291,0,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,0,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,0,1,youu got wild bitches tellin you lies


In [6]:
text['tweet']=text['tweet'].fillna('').astype(str)

In [7]:
x=text['tweet'].copy()
y=text['class'].copy()

In [8]:
x=pd.DataFrame({'ind':x.index, 'text':x.values})


In [9]:
x=x.drop(columns=['ind'])

In [10]:
y=pd.DataFrame(y)

In [11]:
x

,text
0,!!! RT @mayasolovely: As a woman you shouldn't...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,"you've gone and broke the wrong heart baby, an..."
24780,young buck wanna eat!!.. dat nigguh like I ain...
24781,youu got wild bitches tellin you lies


In [12]:
xtrain1,xtest1,ytrain1,ytest1=train_test_split (x['text'], y['class'] , test_size=0.2)
xtest1,xval1,ytest1,yval1=train_test_split (xtest1, ytest1 , test_size=0.5)

In [13]:
xtrain1=pd.DataFrame(xtrain1)
xtest1=pd.DataFrame(xtest1)
xval1=pd.DataFrame(xval1)


In [14]:
xtrain1['class']=ytrain1.copy()
xtest1['class']=ytest1.copy()
xval1['class']=yval1.copy()

In [15]:
xval1

,text,class
11774,Ima kill cay &#128514;&#128557; ugh she stay o...,1
12789,Maybe next time i day drink ill balance it out...,1
23672,bitches love Sosa so I guess I'll wear Sammy S...,1
19873,RT @que: I know u dawg hoes bck in style,1
15075,RT @DuncanIdunno: Smash into windshields? RT @...,2
...,...,...
4837,@T_FLYE lmaoooo I know Taylor in Brazil eating...,1
23895,"happy bro, happy ho: ""Happy husband key to mar...",1
22418,Vice News interview with the Canadian Jihadi h...,2
11491,If I called you a bitCh then you'd prolly hate...,1


In [16]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

handle_encoder = map_name_to_handle[bert_model_name]
handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {handle_encoder}')
print(f'Preprocess model auto-selected: {handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


### Select the Bert encoder and the preprocess model. Here we have the list of all the bert models from tf hub.

In [17]:
bert_preprocess_model = hub.KerasLayer(handle_preprocess)

In [18]:
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_word_ids', 'input_mask', 'input_type_ids']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


### Initiating the bert preprocessor and testing it with some sample text

In [19]:
bert_model = hub.KerasLayer(handle_encoder)

bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Pooled Outputs Shape:(1, 512)
Pooled Outputs Values:[ 0.76262873  0.9928099  -0.18611842  0.36673835  0.15233713  0.6550445
  0.9681154  -0.94862705  0.00216199 -0.9877732   0.06842697 -0.97630596]
Sequence Outputs Shape:(1, 128, 512)
Sequence Outputs Values:[[-0.28946307  0.3432126   0.33231524 ...  0.21300834  0.71020776
  -0.05771159]
 [-0.28742072  0.31980997 -0.23018596 ...  0.5845511  -0.21329741
   0.7269215 ]
 [-0.66157013  0.6887673  -0.8743302  ...  0.10877225 -0.26173237
   0.47855318]
 ...
 [-0.22561178 -0.2892561  -0.07064433 ...  0.4756602   0.83277094
   0.4002539 ]
 [-0.29824233 -0.27473113 -0.05450515 ...  0.48849773  1.0955355
   0.18163365]
 [-0.4437818   0.00930784  0.07223748 ...  0.17290097  1.183325
   0.07897963]]


### Testing the Bert model with preprocessed text

In [90]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  #net = tf.keras.layers.Dense(100, activation='relu')(net)
  net = tf.keras.layers.Dense(3, activation='softmax', name='classifier')(net)
  return tf.keras.Model(text_input, net)

### On top of the pre trained Bert model we add a simple drop out layer and a dense layer for output with softmax activation. We did try other combinations by adding more layers in between but it did'nt give us a good accuracy. so, we sticked to this. We set the 'trainable' parameter to true here. 

In [288]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(['']))
print(tf.sigmoid(bert_raw_result))

tf.Tensor([[0.5158777  0.52894205 0.6943572 ]], shape=(1, 3), dtype=float32)


In [ ]:
tf.keras.utils.plot_model(classifier_model)

In [290]:
loss = tf.keras.losses.CategoricalCrossentropy()
metrics = tf.metrics.CategoricalAccuracy()

### Choosing categorical cross entropy and categorical accuracy as loss and metrics.

In [291]:
epochs = 1
steps_per_epoch = len(xtrain1)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [292]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

### Compiling the model

In [278]:
xtrain1=xtrain1.drop(columns=['class'])
xtest1=xtest1.drop(columns=['class'])
xval1=xval1.drop(columns=['class'])

KeyError: "['class'] not found in axis"

In [86]:
ytrain1=pd.get_dummies(ytrain1)
yval1=pd.get_dummies(yval1)
ytest1=pd.get_dummies(ytest1)

In [87]:
ytrain1.columns=['hatespeech','offensive','nither']
yval1.columns=['hatespeech','offensive','nither']
ytest1.columns=['hatespeech','offensive','nither']

In [279]:
ytest1

,hatespeech,offensive,nither
22708,0,1,0
20679,0,1,0
21595,0,1,0
3315,1,0,0
7758,0,1,0
...,...,...,...
24361,0,1,0
19076,0,1,0
9757,0,1,0
18325,0,0,1


In [293]:
print(f'Training model with {tfhub_handle_encoder}')
new_mod = classifier_model.fit(xtrain1,ytrain1, validation_data= (xval1,yval1), 
                               epochs=epochs)

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
620/620 [==============================] - 4264s 7s/step - loss: 0.5721 - categorical_accuracy: 0.7734 - val_loss: 0.3203 - val_categorical_accuracy: 0.8842


### With just 1 epoch we got the accuracy of 0.8842 

In [285]:
ytest1

,hatespeech,offensive,nither
22708,0,1,0
20679,0,1,0
21595,0,1,0
3315,1,0,0
7758,0,1,0
...,...,...,...
24361,0,1,0
19076,0,1,0
9757,0,1,0
18325,0,0,1


In [38]:
xtrain1=xtrain1.drop(columns=["Maybe next time i day drink ill balance it out..."])

In [46]:
xval1

,text,class
11774,Ima kill cay &#128514;&#128557; ugh she stay o...,1
12789,Maybe next time i day drink ill balance it out...,1
23672,bitches love Sosa so I guess I'll wear Sammy S...,1
19873,RT @que: I know u dawg hoes bck in style,1
15075,RT @DuncanIdunno: Smash into windshields? RT @...,2
...,...,...
4837,@T_FLYE lmaoooo I know Taylor in Brazil eating...,1
23895,"happy bro, happy ho: ""Happy husband key to mar...",1
22418,Vice News interview with the Canadian Jihadi h...,2
11491,If I called you a bitCh then you'd prolly hate...,1


In [295]:
vals=classifier_model.predict(xtest1)

78/78 [==============================] - 142s 2s/step


In [296]:
classifier_model.evaluate(xtest1,ytest1)

78/78 [==============================] - 158s 2s/step - loss: 0.3088 - categorical_accuracy: 0.8914


[0.3087955117225647, 0.891444742679596]

### On testing the model we got teh accuracy of 0.89144

In [299]:
classifier_model.save("C:\\Users\\Dell\\Documents\\projects\\hate speech detection\\hate-speech-and-offensive-language-master\\embeddings\\bert1_lowacc")

In [441]:
classifier_model.predict(["""you look fat"""])

1/1 [==============================] - 0s 465ms/step


array([[0.04771131, 0.73519135, 0.21709739]], dtype=float32)

# loading a saved model

In [442]:
loaded=tf.keras.models.load_model("C:\\Users\\Dell\\Documents\\projects\\hate speech detection\\hate-speech-and-offensive-language-master\\embeddings\\bert1_lowacc",compile=False)

loaded.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)


1/1 [==============================] - 1s 684ms/step


array([[0.13540667, 0.5711486 , 0.2934447 ]], dtype=float32)

In [446]:
loaded.predict([""""""])

1/1 [==============================] - 0s 191ms/step


array([[0.04869141, 0.59821355, 0.35309505]], dtype=float32)